In [1]:
import numpy as np
import pandas as pd
import nltk
import csv
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy import sparse


In [2]:
# 1
df = pd.read_csv('./mapkickstarter/modified_v2.csv')
# df2 = pd.read_csv('./mapkickstarter/word_token.csv')

In [2]:
df = pd.read_csv('./kickstarter-project-statistics/most_backed.csv')

In [92]:
df['word_token'] = df2['text_final']

In [3]:
df2 = pd.DataFrame(columns = ['text_final'])

In [25]:
df2.loc[4,'blurb']

KeyError: 4

In [13]:
df = df[df.status != 'live']

NameError: name 'df' is not defined

In [14]:
df.loc[df.status == 'canceled','status'] = 'failed'
df.loc[df.status == 'suspended','status'] = 'failed'


In [15]:
df['status'] = df['status'].replace({'successful':1,'failed':0})

In [23]:
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
df.reset_index(drop=True,inplace=True)
df


,Unnamed: 0,X1,X1_1,Country,City,id,name,blurb,goal,pledged,...,Backers_as_Prct_of_Pop_Y,Days_spent_making_campign,Days_inception_to_Deadline,Backers_in_city_Y,Backers_in_city_YM,All_Time_Backers_city,Mean_Pledge_City,Mean_pledge_city_Y,Mean_pledge_city_YM,word_token
0,1,1,1,AUS,Adelaide,1337161279,"Worlds & Creatures, the Art of Anthony Christou","[a, 100, page, artbook, featuring, the, work, ...",5000.0,16081.00,...,0.044034,21,59,473,215,1672,3124.000000,2975.532222,215.0,NaN
1,2,2,2,AUS,Adelaide,1533913565,Pheonician Entertainment Productions,"[wordsmith, !, the, story, of, good, vs, evil,...",5500.0,257.00,...,0.044034,22,62,473,5,1672,3124.000000,2975.532222,2.5,NaN
2,3,3,3,AUS,Adelaide,1666009607,The Postnatalist - Second issue comic book,"[the, postnatalist, explores, feminist, issues...",300.0,353.12,...,0.000931,3,48,10,9,1672,3124.000000,94.440000,4.5,NaN
3,4,4,4,AUS,Adelaide,1759296231,Alkandros: Futurewear 3000 Square Sunglasses; ...,"[alkandros, designs, is, interested, in, being...",5000.0,90.00,...,0.044034,84,144,473,3,1672,3124.000000,2975.532222,3.0,NaN
4,6,6,6,AUS,Adelaide,2009794183,E-motive Australia Homemade Rustic Picture Boards,"[homemade, rustic, picture, boards, with, a, s...",4000.0,5.00,...,0.000931,3,33,10,1,1672,3124.000000,94.440000,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88079,99032,99032,99032,USA,Zionsville,552420777,Tabletop Football--Best NEW Tailgating Game fo...,"[patented, tilting, table, technology, makes, ...",55000.0,1179.00,...,0.904208,261,291,101,9,237,2898.166667,1300.000000,9.0,NaN
88080,99033,99033,99033,USA,Zionsville,1440580627,NEW EP by Brett Wiscons - produced by Mark Bry...,"[it, 's, been, a, long, time, coming, ,, but, ...",8400.0,11130.00,...,0.760967,75,108,85,85,237,2898.166667,11130.000000,85.0,NaN
88081,99034,99034,99034,USA,Zionsville,1578043977,Alzheimer's Stress Relief: How to Make Difficu...,"[a, practical, guide, to, help, caregivers, ma...",3000.0,586.00,...,0.456580,9,39,51,13,237,2898.166667,1179.500000,13.0,NaN
88082,99035,99035,99035,USA,Zionsville,1664889418,The Local Flavor EP by Brett Wiscons,"[indiana, 's, brett, wiscons, is, back, with, ...",3150.0,530.00,...,0.904208,12,32,101,11,237,2898.166667,1300.000000,11.0,NaN


In [3]:
df['blurb'] = [str(entry).lower() for entry in df['blurb']]
df['blurb'] = [word_tokenize(entry) for entry in df['blurb']]
df['blurb']
# with open('./mapkickstarter/MasterKickstarter.csv','r') as csv_file:
#     csv_reader = csv.reader(csv_file,delimiter=',')
#     next(csv_reader)
#     with open('output.csv','a') as out:
#         csv_writer = csv.writer(out,lineterminator='\n')
#         csv_writer.writerow('final_text')
#         for lines in csv_reader:
#             csv_writer.writerow(['test','test'])
            


0       [this, is, a, card, game, for, people, who, ar...
1       [an, unusually, addicting, ,, high-quality, de...
2       [bring, reading, rainbow, ’, s, library, of, i...
3       [updated, :, this, is, it, ., we, 're, making,...
4       [an, adventure, game, from, tim, schafer, ,, d...
                              ...                        
3995    [become, a, monster, to, fight, the, monsters—...
3996    [the, gobone, automatically, entertains, your,...
3997    [create, and, share, better, spherical, panora...
3998    [the, origins, returned, to, the, 16th, centur...
3999    [52, weeks, ., 52, artists, ., one, illustrate...
Name: blurb, Length: 4000, dtype: object

In [4]:
# WordNetLemmatizer requires Pos tags to understand if the word is 
# noun or verb or adjective etc. By default it is set to Noun
# source: https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
# with open('./mapkickstarter/modified.csv','r') as csv_file:
#     csv_reader = csv.reader(csv_file,delimiter=',')
#     next(csv_reader)
#     with open('output.csv','a') as out:
#         header = []
#         csv_writer = csv.writer(out,lineterminator='\n')
#         header.append('final_text')
#         csv_writer.writerow(header)
#         for lines in csv_reader:
#             Final_words = []
#             word_Lemmatized = WordNetLemmatizer()
#             for word, tag in pos_tag(lines[7]):
#                 if word not in stopwords.words('english') and word.isalpha():
#                     word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
#                     Final_words.append(word_Final)
#             csv_writer.writerow(Final_words)

for index,entry in enumerate(df['blurb']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'word_token'] = str(Final_words)
    print('progress: ',index)

progress:  0
progress:  1
progress:  2
progress:  3
progress:  4
progress:  5
progress:  6
progress:  7
progress:  8
progress:  9
progress:  10
progress:  11
progress:  12
progress:  13
progress:  14
progress:  15
progress:  16
progress:  17
progress:  18
progress:  19
progress:  20
progress:  21
progress:  22
progress:  23
progress:  24
progress:  25
progress:  26
progress:  27
progress:  28
progress:  29
progress:  30
progress:  31
progress:  32
progress:  33
progress:  34
progress:  35
progress:  36
progress:  37
progress:  38
progress:  39
progress:  40
progress:  41
progress:  42
progress:  43
progress:  44
progress:  45
progress:  46
progress:  47
progress:  48
progress:  49
progress:  50
progress:  51
progress:  52
progress:  53
progress:  54
progress:  55
progress:  56
progress:  57
progress:  58
progress:  59
progress:  60
progress:  61
progress:  62
progress:  63
progress:  64
progress:  65
progress:  66
progress:  67
progress:  68
progress:  69
progress:  70
progress:  71
pr

progress:  557
progress:  558
progress:  559
progress:  560
progress:  561
progress:  562
progress:  563
progress:  564
progress:  565
progress:  566
progress:  567
progress:  568
progress:  569
progress:  570
progress:  571
progress:  572
progress:  573
progress:  574
progress:  575
progress:  576
progress:  577
progress:  578
progress:  579
progress:  580
progress:  581
progress:  582
progress:  583
progress:  584
progress:  585
progress:  586
progress:  587
progress:  588
progress:  589
progress:  590
progress:  591
progress:  592
progress:  593
progress:  594
progress:  595
progress:  596
progress:  597
progress:  598
progress:  599
progress:  600
progress:  601
progress:  602
progress:  603
progress:  604
progress:  605
progress:  606
progress:  607
progress:  608
progress:  609
progress:  610
progress:  611
progress:  612
progress:  613
progress:  614
progress:  615
progress:  616
progress:  617
progress:  618
progress:  619
progress:  620
progress:  621
progress:  622
progress: 

progress:  1106
progress:  1107
progress:  1108
progress:  1109
progress:  1110
progress:  1111
progress:  1112
progress:  1113
progress:  1114
progress:  1115
progress:  1116
progress:  1117
progress:  1118
progress:  1119
progress:  1120
progress:  1121
progress:  1122
progress:  1123
progress:  1124
progress:  1125
progress:  1126
progress:  1127
progress:  1128
progress:  1129
progress:  1130
progress:  1131
progress:  1132
progress:  1133
progress:  1134
progress:  1135
progress:  1136
progress:  1137
progress:  1138
progress:  1139
progress:  1140
progress:  1141
progress:  1142
progress:  1143
progress:  1144
progress:  1145
progress:  1146
progress:  1147
progress:  1148
progress:  1149
progress:  1150
progress:  1151
progress:  1152
progress:  1153
progress:  1154
progress:  1155
progress:  1156
progress:  1157
progress:  1158
progress:  1159
progress:  1160
progress:  1161
progress:  1162
progress:  1163
progress:  1164
progress:  1165
progress:  1166
progress:  1167
progress

progress:  1630
progress:  1631
progress:  1632
progress:  1633
progress:  1634
progress:  1635
progress:  1636
progress:  1637
progress:  1638
progress:  1639
progress:  1640
progress:  1641
progress:  1642
progress:  1643
progress:  1644
progress:  1645
progress:  1646
progress:  1647
progress:  1648
progress:  1649
progress:  1650
progress:  1651
progress:  1652
progress:  1653
progress:  1654
progress:  1655
progress:  1656
progress:  1657
progress:  1658
progress:  1659
progress:  1660
progress:  1661
progress:  1662
progress:  1663
progress:  1664
progress:  1665
progress:  1666
progress:  1667
progress:  1668
progress:  1669
progress:  1670
progress:  1671
progress:  1672
progress:  1673
progress:  1674
progress:  1675
progress:  1676
progress:  1677
progress:  1678
progress:  1679
progress:  1680
progress:  1681
progress:  1682
progress:  1683
progress:  1684
progress:  1685
progress:  1686
progress:  1687
progress:  1688
progress:  1689
progress:  1690
progress:  1691
progress

progress:  2150
progress:  2151
progress:  2152
progress:  2153
progress:  2154
progress:  2155
progress:  2156
progress:  2157
progress:  2158
progress:  2159
progress:  2160
progress:  2161
progress:  2162
progress:  2163
progress:  2164
progress:  2165
progress:  2166
progress:  2167
progress:  2168
progress:  2169
progress:  2170
progress:  2171
progress:  2172
progress:  2173
progress:  2174
progress:  2175
progress:  2176
progress:  2177
progress:  2178
progress:  2179
progress:  2180
progress:  2181
progress:  2182
progress:  2183
progress:  2184
progress:  2185
progress:  2186
progress:  2187
progress:  2188
progress:  2189
progress:  2190
progress:  2191
progress:  2192
progress:  2193
progress:  2194
progress:  2195
progress:  2196
progress:  2197
progress:  2198
progress:  2199
progress:  2200
progress:  2201
progress:  2202
progress:  2203
progress:  2204
progress:  2205
progress:  2206
progress:  2207
progress:  2208
progress:  2209
progress:  2210
progress:  2211
progress

progress:  2667
progress:  2668
progress:  2669
progress:  2670
progress:  2671
progress:  2672
progress:  2673
progress:  2674
progress:  2675
progress:  2676
progress:  2677
progress:  2678
progress:  2679
progress:  2680
progress:  2681
progress:  2682
progress:  2683
progress:  2684
progress:  2685
progress:  2686
progress:  2687
progress:  2688
progress:  2689
progress:  2690
progress:  2691
progress:  2692
progress:  2693
progress:  2694
progress:  2695
progress:  2696
progress:  2697
progress:  2698
progress:  2699
progress:  2700
progress:  2701
progress:  2702
progress:  2703
progress:  2704
progress:  2705
progress:  2706
progress:  2707
progress:  2708
progress:  2709
progress:  2710
progress:  2711
progress:  2712
progress:  2713
progress:  2714
progress:  2715
progress:  2716
progress:  2717
progress:  2718
progress:  2719
progress:  2720
progress:  2721
progress:  2722
progress:  2723
progress:  2724
progress:  2725
progress:  2726
progress:  2727
progress:  2728
progress

progress:  3182
progress:  3183
progress:  3184
progress:  3185
progress:  3186
progress:  3187
progress:  3188
progress:  3189
progress:  3190
progress:  3191
progress:  3192
progress:  3193
progress:  3194
progress:  3195
progress:  3196
progress:  3197
progress:  3198
progress:  3199
progress:  3200
progress:  3201
progress:  3202
progress:  3203
progress:  3204
progress:  3205
progress:  3206
progress:  3207
progress:  3208
progress:  3209
progress:  3210
progress:  3211
progress:  3212
progress:  3213
progress:  3214
progress:  3215
progress:  3216
progress:  3217
progress:  3218
progress:  3219
progress:  3220
progress:  3221
progress:  3222
progress:  3223
progress:  3224
progress:  3225
progress:  3226
progress:  3227
progress:  3228
progress:  3229
progress:  3230
progress:  3231
progress:  3232
progress:  3233
progress:  3234
progress:  3235
progress:  3236
progress:  3237
progress:  3238
progress:  3239
progress:  3240
progress:  3241
progress:  3242
progress:  3243
progress

progress:  3696
progress:  3697
progress:  3698
progress:  3699
progress:  3700
progress:  3701
progress:  3702
progress:  3703
progress:  3704
progress:  3705
progress:  3706
progress:  3707
progress:  3708
progress:  3709
progress:  3710
progress:  3711
progress:  3712
progress:  3713
progress:  3714
progress:  3715
progress:  3716
progress:  3717
progress:  3718
progress:  3719
progress:  3720
progress:  3721
progress:  3722
progress:  3723
progress:  3724
progress:  3725
progress:  3726
progress:  3727
progress:  3728
progress:  3729
progress:  3730
progress:  3731
progress:  3732
progress:  3733
progress:  3734
progress:  3735
progress:  3736
progress:  3737
progress:  3738
progress:  3739
progress:  3740
progress:  3741
progress:  3742
progress:  3743
progress:  3744
progress:  3745
progress:  3746
progress:  3747
progress:  3748
progress:  3749
progress:  3750
progress:  3751
progress:  3752
progress:  3753
progress:  3754
progress:  3755
progress:  3756
progress:  3757
progress

In [44]:
df['launched_at'] = pd.to_datetime(df['launched_at'],yearfirst=True)
df['launched_at']

0       2017-05-29
1       2017-06-22
2       2014-12-12
3       2017-02-28
4       2014-06-16
           ...    
88079   2016-09-08
88080   2013-04-06
88081   2014-07-30
88082   2016-03-27
88083   2016-11-13
Name: launched_at, Length: 88084, dtype: datetime64[ns]

In [45]:
df['deadline'] = pd.to_datetime(df['deadline'],yearfirst=True)
df['deadline']

0       2017-07-06
1       2017-08-01
2       2015-01-26
3       2017-04-29
4       2014-07-16
           ...    
88079   2016-10-08
88080   2013-05-09
88081   2014-08-29
88082   2016-04-16
88083   2016-12-13
Name: deadline, Length: 88084, dtype: datetime64[ns]

In [46]:
df['deadline'] = (df['deadline'] - df['launched_at']).dt.days
df['deadline']

0        38
1        40
2        45
3        60
4        30
         ..
88079    30
88080    33
88081    30
88082    20
88083    30
Name: deadline, Length: 88084, dtype: int64

In [ ]:
df.to_csv('./mapkickstarter/modified_v2.csv',header = True, index=False)

In [2]:
# 2
df['word_token'] = df['word_token'].astype(str)

NameError: name 'df' is not defined

In [8]:
# 3
df2 = df[['goal','Length_of_kick','status']]


In [11]:
# 4
tfidf_vect = TfidfVectorizer(max_features = 5000)
tfidf_vect.fit(df['word_token'])
x_tfidf = tfidf_vect.transform(df['word_token'])

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_features = 5000)
count_vect.fit(df['word_token'])
x_count = tfidf_vect.transform(df['word_token'])


In [13]:
import pickle
filename = 'word_vect.sav'
pickle.dump(count_vect, open(filename, 'wb'))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle 
count_vect = pickle.load(open('word_vect.sav','rb'))
x_count = count_vect.transform(df['word_token'])

In [7]:
# 5
df3 = pd.DataFrame(x_count)

In [11]:
# 6
df4 = pd.concat([df2,df3],axis=1)

In [12]:
x = df4[['goal','Length_of_kick']].values
y = df4['status'].values
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)

In [9]:
# 7
x_1 = df4[['goal']].values
x_2 = df4[['Length_of_kick']].values
y = df4[['status']].values

In [10]:
# 8
training_data = sparse.hstack((x_count,x_1,x_2))

In [14]:
# 9
training_data.data = np.nan_to_num(training_data.data)

In [11]:
training_data.data.shape

(938649,)

In [15]:
# 10
training_data.eliminate_zeros()

In [18]:
training_data.data.shape

(1017711,)

In [16]:
# 11
x_train, x_test, y_train, y_test = train_test_split(training_data,y,test_size =0.2)

In [18]:
x_train

<70467x5002 sparse matrix of type '<class 'numpy.float64'>'
	with 814033 stored elements in Compressed Sparse Row format>

In [26]:
SVM
clf = svm.SVC()
clf.fit(training_data,y)
predicted = clf.predict(training_data)
print(predicted)

tn, fp, fn, tp = metrics.confusion_matrix(y, predicted).ravel()
print("tn fp fn tp")
print(tn, fp, fn, tp)
print("recall: ", (tp/(tp+fn)))
print("precise: ", (tp/(tp+fp)))
print(clf.score(x_test, y_test))


In [28]:
# neural network
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(300,150,150),activation='relu')
mlp.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300, 150, 150), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [30]:
# predicted = (mlp.predict_proba(x_test)[:,1] >= 0.415).astype(int)
predicted = mlp.predict_proba(x_test)
print(predicted)
# tn, fp, fn, tp = metrics.confusion_matrix(y_test, predicted).ravel()
# print("tn fp fn tp")
# print(tn, fp, fn, tp)
# print("recall: ", (tp/(tp+fn)))
# print("precise: ", (tp/(tp+fp)))
# print(mlp.score(x_test, y_test))

[[0.66396048 0.33603952]
 [0.39571545 0.60428455]
 [0.46393818 0.53606182]
 ...
 [0.96429677 0.03570323]
 [0.52482644 0.47517356]
 [0.62122235 0.37877765]]


In [31]:
file_name = 'mlp_model.sav'
pickle.dump(mlp,open(file_name,'wb'))

In [22]:
print(tfidf_vect.vocabulary_)

{u'cycling': 1088, u'opener': 3114, u'wooden': 4942, u'undead': 4680, u'cooking': 973, u'china': 756, u'uplifting': 4725, u'golden': 1930, u'projection': 3496, u'dna': 1282, u'catchy': 694, u'music': 2951, u'titanium': 4535, u'want': 4839, u'absolute': 7, u'travel': 4606, u'wrong': 4972, u'fit': 1731, u'songwriter': 4166, u'mason': 2749, u'encourage': 1467, u'adapt': 50, u'gameplay': 1864, u'specially': 4197, u'service': 3998, u'master': 2752, u'genesis': 1890, u'task': 4422, u'positively': 3395, u'feeling': 1681, u'spectrum': 4203, u'dozen': 1316, u'wholesome': 4898, u'mostly': 2921, u'cinematic': 777, u'simplify': 4075, u'mouth': 2933, u'singer': 4083, u'tech': 4438, u'scream': 3933, u'orleans': 3145, u'rick': 3793, u'rich': 3790, u'rice': 3789, u'plate': 3335, u'patch': 3221, u'heirloom': 2082, u'extend': 1611, u'nature': 2985, u'angel': 178, u'union': 4693, u'much': 2938, u'fry': 1829, u'dave': 1118, u'spin': 4213, u'professionally': 3484, u'employ': 1453, u'miniature': 2856, u'rec

In [23]:
df['text_final']

KeyError: 'text_final'

In [24]:
df['status'].value_counts()

0    51125
1    36959
Name: status, dtype: int64